# Generate Training and Testing files, as well as doing trend analysis

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import csv

In [184]:
# icd9 for fever identification
fever_icd9 = ['7806']
# icd9s for neutropenia identification (reference: "Fever and neutropenia hospital discharges in children with cancer: A 2012 update")
# neu_icd9 = ['2880', '2841', '1125']
neu_icd9 = ['2880']
# ccs for cancer identification
cancer_ccs = list(range(11,46))
cancer_ccs = [str(x) for x in cancer_ccs]
# path of the file of all discharges of a certain year
path = '../../../data/hcup/nis/NIS_2015/total.csv'

In [185]:
# load all discharges of a particular year
data = pd.read_csv(path, dtype = object, quoting = csv.QUOTE_ALL)

In [186]:
data.reset_index(drop = True, inplace = True)

## Load trend weight for years before 2012 (for 2012 or later, use DISCWT directly)

reference: https://www.hcup-us.ahrq.gov/reports/methods/2006_05_NISTrendsReport_1988-2004.pdf

#### DO NOT RUN THE FOLLOWING CELLS FOR 2012 OR LATER YEARS

In [114]:
# load trend weights file from url
target_url = 'https://www.hcup-us.ahrq.gov/db/nation/nis/weights/NIS_2011_HOSPITAL_TrendWt.ASC'
import urllib
content=urllib.request.urlopen(target_url)
hospids = []
trendwts = []
for line in content:
    # hospital id
    hospids.append(int(line[4:10].strip()))
    # trend weight 
    trendwts.append(float(line[10:].strip()))

In [115]:
def get_trendwt(row):
    return trendwts[hospids.index(int(row['HOSPID']))]

In [116]:
data['DISCWT'] = data.apply(lambda row: get_trendwt(row),axis=1)

#### THE FOLLOWING CELL IS ONLY FOR 2015

In [187]:
# For 2015, there is only one core file and one hospital file, but two severity files and two DX_PR_GRPS files;
# since we do not use Q4, we only need one severity file and one DX_PR_GRPS file. So, after merging Q1Q3's severity/DX_PR_GRPS file,
# there would be some redundancy in the merged file (i.e. some discharges belongs to Q4). Those redundancies would have Nan values.
# So, remove them.

data.dropna(subset = ['CM_LYMPH', 'CM_LIVER', 'CM_PARA', 'CM_RENLFAIL', 'CM_AIDS', 'CM_ALCOHOL', 'CM_ANEMDEF',
            'CM_ARTH', 'CM_BLDLOSS', 'CM_CHF', 'CM_CHRNLUNG', 'CM_COAG', 'CM_DEPRESS', 'CM_DM', 'CM_DMCX', 'CM_DRUG',
            'CM_HTN_C', 'CM_HYPOTHY', 'CM_LYTES', 'CM_METS', 'CM_OBESE', 'CM_NEURO', 'CM_PERIVASC', 'CM_PSYCH',
            'CM_PULMCIRC', 'CM_ULCER', 'CM_VALVE', 'CM_WGHTLOSS', 'CM_TUMOR', 'APRDRG', 'APRDRG_Severity', 'APRDRG_Risk_Mortality'], 
               how = 'all', inplace = True)

#### RUN THE FOLLOWING CELLS FOR ALL YEARS DATA

In [188]:
len(data)

5377720

In [189]:
# The output of this cell can be verified with HCUPnet (https://hcupnet.ahrq.gov/#setup)

np.sum(data.DISCWT.values.astype(float))

26888597.691980608

In [190]:
# missing or invalid/inconsistant data are represented by alphabats or negative numbers

data_missingage = data[(data.AGE.isnull()) |(data.AGE=='A') | (data.AGE=='B') | (data.AGE=='C') | (data.AGE < '0')]
data = data.drop(index = data_missingage.index)
data.AGE = data.AGE.astype(float)
data_adult = data[data.AGE>=18]
data_young = data[data.AGE<18]

In [191]:
print('number of adult discharges: ', np.sum(data_adult.DISCWT.values.astype(float)))
print('number of young discharges: ', np.sum(data_young.DISCWT.values.astype(float)))
print('number of missing age discharges: ', np.sum(data_missingage.DISCWT.values.astype(float)))

number of adult discharges:  22682217.71335061
number of young discharges:  4193790.0971502014
number of missing age discharges:  12589.8814798


In [192]:
len(data)

5375202

In [193]:
len(data_missingage)

2518

In [194]:
# columns for diagnoses; for years before 2009, there are 15 columns, there are 25 for 2009-2013, 30 for 2014 and 2015

dx_cols = ['DX1',
 'DX2',
 'DX3',
 'DX4',
 'DX5',
 'DX6',
 'DX7',
 'DX8',
 'DX9',
 'DX10',
 'DX11',
 'DX12',
 'DX13',
 'DX14',
 'DX15',
 'DX16',
 'DX17',
 'DX18',
 'DX19',
 'DX20',
 'DX21',
 'DX22',
 'DX23',
 'DX24',
 'DX25',
 'DX26',
 'DX27',
 'DX28',
 'DX29',
 'DX30']

In [195]:
# columns for CCS; for years before 2009, there are 15 columns, there are 25 for 2009-2013, 30 for 2014 and 2015

ccs_cols = ['DXCCS1',
 'DXCCS2',
 'DXCCS3',
 'DXCCS4',
 'DXCCS5',
 'DXCCS6',
 'DXCCS7',
 'DXCCS8',
 'DXCCS9',
 'DXCCS10',
 'DXCCS11',
 'DXCCS12',
 'DXCCS13',
 'DXCCS14',
 'DXCCS15',
 'DXCCS16',
 'DXCCS17',
 'DXCCS18',
 'DXCCS19',
 'DXCCS20',
 'DXCCS21',
 'DXCCS22',
 'DXCCS23',
 'DXCCS24',
 'DXCCS25',
 'DXCCS26',
 'DXCCS27',
 'DXCCS28',
 'DXCCS29',
 'DXCCS30']

In [196]:
# columns for chronic condition identifier; for years before 2009, there are 15 columns, there are 25 for 2009-2013, 30 for 2014 and 2015

chron_cols = ['CHRON1',
 'CHRON2',
 'CHRON3',
 'CHRON4',
 'CHRON5',
 'CHRON6',
 'CHRON7',
 'CHRON8',
 'CHRON9',
 'CHRON10',
 'CHRON11',
 'CHRON12',
 'CHRON13',
 'CHRON14',
 'CHRON15',
 'CHRON16',
 'CHRON17',
 'CHRON18',
 'CHRON19',
 'CHRON20',
 'CHRON21',
 'CHRON22',
 'CHRON23',
 'CHRON24',
 'CHRON25',
 'CHRON26',
 'CHRON27',
 'CHRON28',
 'CHRON29',
 'CHRON30']

In [197]:
# function for selecting discharges with certain icd9 codes

def icd9_selection(df, cols, icd9s):   
    pt = pd.DataFrame(data=None, columns=df.columns)   
    for col in cols:        
        for icd9 in icd9s:        
            pt = pt.append(df[df[col].str.startswith(icd9, na = False)])
    return pt.drop_duplicates()

In [198]:
# function for selecting discharges with certain CCS codes

def ccs_selection(df, cols, ccs_codes):   
    pt = pd.DataFrame(data=None, columns=df.columns)   
    for col in cols:       
        pt = pt.append(df[df[col].isin(cancer_ccs)])
    return pt.drop_duplicates()

#### Stats for cancer admissions

In [199]:
cancer = ccs_selection(data, ccs_cols, cancer_ccs)
cancer_adult = ccs_selection(data_adult, ccs_cols, cancer_ccs)
cancer_young = ccs_selection(data_young, ccs_cols, cancer_ccs)
cancer_missing = ccs_selection(data_missingage, ccs_cols, cancer_ccs)
print('number of adult cancer patient: ', np.sum(cancer_adult.DISCWT.values.astype(float)))
print('number of young cancer patient: ', np.sum(cancer_young.DISCWT.values.astype(float)))
print('number of missing age cancer patient: ', np.sum(cancer_missing.DISCWT.values.astype(float)))

number of adult cancer patient:  3737865.228388001
number of young cancer patient:  76954.9916835
number of missing age cancer patient:  424.9968154


#### Stats for fever and neutropenia admissions

In [49]:
fever_adult = icd9_selection(cancer_adult, dx_cols, fever_icd9)
fever_young = icd9_selection(cancer_young, dx_cols, fever_icd9)
fever_missing = icd9_selection(cancer_missing, dx_cols, fever_icd9)
neu_adult = icd9_selection(cancer_adult, dx_cols, neu_icd9)
neu_young = icd9_selection(cancer_young, dx_cols, neu_icd9)
neu_missing = icd9_selection(cancer_missing, dx_cols, neu_icd9)
print('number of adult fever patient: ', np.sum(fever_adult.DISCWT.values.astype(float)))
print('number of young fever patient: ', np.sum(fever_young.DISCWT.values.astype(float)))
print('number of missing fever patient: ', np.sum(fever_missing.DISCWT.values.astype(float)))
print('number of adult Neutraphenia patient: ', np.sum(neu_adult.DISCWT.values.astype(float)))
print('number of young Neutraphenia patient: ', np.sum(neu_young.DISCWT.values.astype(float)))
print('number of missing Neutraphenia patient: ', np.sum(neu_missing.DISCWT.values.astype(float)))

number of adult fever patient:  127774.59735640002
number of young fever patient:  18763.5816579
number of missing fever patient:  3.138045
number of adult Neutraphenia patient:  109378.84070260002
number of young Neutraphenia patient:  17117.301849099997
number of missing Neutraphenia patient:  3.77492


#### Stats for febrile neutropenia admissions

In [50]:
#fn_pt = pd.merge(fever_pt, neu_pt, how = 'inner')
#no_fn_pt = len(fn_pt)
#print(fn_pt.iloc[0:3])
fn_adult = icd9_selection(fever_adult, dx_cols, neu_icd9)
fn_young = icd9_selection(fever_young, dx_cols, neu_icd9)
fn_missing = icd9_selection(fever_missing, dx_cols, neu_icd9)

#no_fn_pt = len(fn_pt)
#no_fn_pt_2 = len(fn_pt_2)
print('number of adult Febrile Neutraphenia patient: ', np.sum(fn_adult.DISCWT.values.astype(float)))
print('number of young Febrile Neutraphenia patient: ', np.sum(fn_young.DISCWT.values.astype(float)))
print('number of missing Febrile Neutraphenia patient: ', np.sum(fn_missing.DISCWT.values.astype(float)))

number of adult Febrile Neutraphenia patient:  56406.799010400005
number of young Febrile Neutraphenia patient:  12000.6342123
number of missing Febrile Neutraphenia patient:  0.0


In [51]:
len(fn_adult)

11662

#### Stats for died febrile neutropenia admissions

In [52]:
#fn_pt = pd.merge(fever_pt, neu_pt, how = 'inner')
#no_fn_pt = len(fn_pt)
#print(fn_pt.iloc[0:3])
fn_adult_died = fn_adult[fn_adult.DIED == '1']
fn_young_died = fn_young[fn_young.DIED == '1']
fn_missing_died = fn_missing[fn_missing.DIED == '1']

#no_fn_pt = len(fn_pt)
#no_fn_pt_2 = len(fn_pt_2)
print('number of died adult Febrile Neutraphenia patient: ', np.sum(fn_adult_died.DISCWT.values.astype(float)))
print('number of died young Febrile Neutraphenia patient: ', np.sum(fn_young_died.DISCWT.values.astype(float)))
print('number of died missing Febrile Neutraphenia patient: ', np.sum(fn_missing_died.DISCWT.values.astype(float)))

number of died adult Febrile Neutraphenia patient:  1908.7448290999998
number of died young Febrile Neutraphenia patient:  60.4881243
number of died missing Febrile Neutraphenia patient:  0.0


## Generate training and testing data for FN admissions

In [53]:
# identify target variable

target_vars = ['DIED']

In [54]:
# remove discharges with missing diad information

fn_adult_withdied=fn_adult[fn_adult.DIED.isin(['0','1'])]

In [55]:
fn_adult_missingdied=fn_adult[~fn_adult.DIED.isin(['0','1'])]

In [56]:
len(fn_adult_missingdied)

2

#### RUN THE FOLLOWING CELL ONLY FOR 2007

In [57]:
# for 2007, run this cell to create NCHRONIC (number of chronic conditions) variable, since 2007 do not have it.
# reference: https://www.hcup-us.ahrq.gov/db/vars/nchronic/nisnote.jsp

def NCHRONIC(row):
    ccs=[]
    n = 0
    for i in range(15):
        if row[chron_cols[i]] == '1' and row[ccs_cols[i]] not in ccs:
            n += 1
            ccs.append(row[ccs_cols[i]])
    return n
fn_adult['NCHRONIC'] = fn_adult.apply(lambda row: NCHRONIC(row),axis=1)

#### RUN THE FOLLOWING CELLS FOR ALL YEAR DATA

In [58]:
train_x, val_x, _, _ = train_test_split(fn_adult, fn_adult.DIED, test_size=0.2, random_state=None, shuffle = True)

In [200]:
# the output file paths

path_fn_adult_missingdied = '../../../data/hcup/nis/2007/fn_adult_missingdied_2007.csv'
path_data_withage = '../../../data/hcup/nis/2007/total_withage_2007.csv'
path_data_missingage = '../../../data/hcup/nis/2007/total_missingage_2007.csv'
path_cancer = '../../../data/hcup/nis/NIS_2015/cancer_2015.csv'
path_train_and_test = '../../../data/hcup/nis/2007/train_test_2007.csv'
path_train = '../../../data/hcup/nis/2007/train_2007.csv'
path_test = '../../../data/hcup/nis/2007/test_2007.csv'

In [60]:
fn_adult_missingdied.to_csv(path_fn_adult_missingdied, quoting = csv.QUOTE_ALL, index = False)

In [61]:
data.to_csv(path_data_withage, quoting = csv.QUOTE_ALL, index = False)

In [77]:
data_missingage.to_csv(path_data_missingage, quoting = csv.QUOTE_ALL, index = False)

In [201]:
cancer_adult.to_csv(path_cancer, quoting = csv.QUOTE_ALL, index = False)

In [63]:
fn_adult.to_csv(path_train_and_test, quoting = csv.QUOTE_ALL, index = False)

In [64]:
train_x.to_csv(path_train, quoting = csv.QUOTE_ALL, index = False)

In [65]:
val_x.to_csv(path_test, quoting = csv.QUOTE_ALL, index = False)